# Libraries

In [ ]:
# !pip install ipympl
%matplotlib widget 
# makes sure plots are saved in folder and not displayed in notebook

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, make_scorer

# Confusion Matrix Function

In [ ]:
# Plot confusion matrix
def cm_plot(cm_value, model, total=False):
    classes = ['Low mental workload', 'High mental workload']
    plt.figure(figsize=(8, 6))
    plt.imshow(cm_value, interpolation='nearest', cmap='coolwarm')
    plt.title(f'Confusion Matrix - {model}')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')

    # Add the values to the confusion matrix plot
    thresh = cm_value.max() / 2.0
    for i in range(cm_value.shape[0]):
        for j in range(cm_value.shape[1]):
            plt.text(j, i, format(cm_value[i, j], 'd'), ha='center', va='center',
                     color='white' if cm_value[i, j] > thresh else 'black')

    plt.tight_layout()
    plt.show()
    output_file = f'/Flight-Sim-Cognitive-Workload-EEG-Prediction/results/confusion_matrices/cmplot_{model}.png'
    plt.savefig(output_file, bbox_inches='tight')
    plt.close()

# Fronto-Parietal Phase Locking Value (PLV) Connectivity function
### Retrieving the average and standardized version of the Fronto-Parietal Phase Locking Value (PLV) Connectivity features

In [ ]:
def fronto_parietal_averagePLV(data):
    '''
    Computes the mean connectivity between frontal and parietal channels bidirectionally.
    
    Parameters:
        data: Connectivity data for each channel.
    
    Returns:
        numpy.ndarray: (average) fronto-parietal connectivity values for each channel and for each frequency band (alpha, beta, theta).
    '''
    
    # Alpha mean PLV values
    F7_A = np.mean(data[4:9])
    F3_A = np.mean(data[12:17])
    Fz_A = np.mean(data[19:24])
    F4_A = np.mean(data[25:30])
    F8_A = np.mean(data[30:35])
    P3_A = np.mean(np.concatenate((data[4:5], data[12:13], data[19:20], data[25:26], data[30:31])))
    Pz_A = np.mean(np.concatenate((data[5:6], data[13:14], data[20:21], data[26:27], data[31:32])))
    P4_A = np.mean(np.concatenate((data[6:7], data[14:15], data[21:22], data[27:28], data[32:33])))
    PO7_A = np.mean(np.concatenate((data[7:8], data[15:16], data[22:23], data[28:29], data[33:34])))
    PO8_A = np.mean(np.concatenate((data[8:9], data[16:17], data[23:24], data[29:30], data[34:35])))
    
    # Beta mean PLV values
    F7_B = np.mean(data[4+45:9+45])
    F3_B = np.mean(data[12+45:17+45])
    Fz_B = np.mean(data[19+45:24+45])
    F4_B = np.mean(data[25+45:30+45])
    F8_B = np.mean(data[30+45:35+45])
    P3_B = np.mean(np.concatenate((data[4+45:5+45], data[12+45:13+45], data[19+45:20+45], data[25+45:26+45], data[30+45:31+45])))
    Pz_B = np.mean(np.concatenate((data[5+45:6+45], data[13+45:14+45], data[20+45:21+45], data[26+45:27+45], data[31+45:32+45])))
    P4_B = np.mean(np.concatenate((data[6+45:7+45], data[14+45:15+45], data[21+45:22+45], data[27+45:28+45], data[32+45:33+45])))
    PO7_B = np.mean(np.concatenate((data[7+45:8+45], data[15+45:16+45], data[22+45:23+45], data[28+45:29+45], data[33+45:34+45])))
    PO8_B = np.mean(np.concatenate((data[8+45:9+45], data[16+45:17+45], data[23+45:24+45], data[29+45:30+45], data[34+45:35+45])))
    
    # Theta mean PLV values
    F7_T = np.mean(data[4+90:9+90])
    F3_T = np.mean(data[12+90:17+90])
    Fz_T = np.mean(data[19+90:24+90])
    F4_T = np.mean(data[25+90:30+90])
    F8_T = np.mean(data[30+90:35+90])
    P3_T = np.mean(np.concatenate((data[4+90:5+90], data[12+90:13+90], data[19+90:20+90], data[25+90:26+90], data[30+90:31+90])))
    Pz_T = np.mean(np.concatenate((data[5+90:6+90], data[13+90:14+90], data[20+90:21+90], data[26+90:27+90], data[31+90:32+90])))
    P4_T = np.mean(np.concatenate((data[6+90:7+90], data[14+90:15+90], data[21+90:22+90], data[27+90:28+90], data[32+90:33+90])))
    PO7_T = np.mean(np.concatenate((data[7+90:8+90], data[15+90:16+90], data[22+90:23+90], data[28+90:29+90], data[33+90:34+90])))
    PO8_T = np.mean(np.concatenate((data[8+90:9+90], data[16+90:17+90], data[23+90:24+90], data[29+90:30+90], data[34+90:35+90])))
    
    min_value = np.min([F7_A, F3_A, Fz_A, F4_A, F8_A, P3_A, Pz_A, P4_A, PO7_A, PO8_A, F7_B, F3_B, Fz_B, F4_B, F8_B, P3_B, Pz_B, P4_B, PO7_B, PO8_B, F7_T, F3_T, Fz_T, F4_T, F8_T, P3_T, Pz_T, P4_T, PO7_T, PO8_T])
    max_value = np.max([F7_A, F3_A, Fz_A, F4_A, F8_A, P3_A, Pz_A, P4_A, PO7_A, PO8_A, F7_B, F3_B, Fz_B, F4_B, F8_B, P3_B, Pz_B, P4_B, PO7_B, PO8_B, F7_T, F3_T, Fz_T, F4_T, F8_T, P3_T, Pz_T, P4_T, PO7_T, PO8_T])
    
    PLVs_alpha = [(x - min_value) / (max_value - min_value) * 2 - 1 for x in [F7_A, F3_A, Fz_A, F4_A, F8_A, P3_A, Pz_A, P4_A, PO7_A, PO8_A]]    
    PLVs_theta = [(x - min_value) / (max_value - min_value) * 2 - 1 for x in [F7_B, F3_B, Fz_B, F4_B, F8_B, P3_B, Pz_B, P4_B, PO7_B, PO8_B]]
    PLVs_beta = [(x - min_value) / (max_value - min_value) * 2 - 1 for x in [F7_T, F3_T, Fz_T, F4_T, F8_T, P3_T, Pz_T, P4_T, PO7_T, PO8_T]]
    
    data_x = np.concatenate((PLVs_alpha, PLVs_beta, PLVs_theta))

    return data_x


# Baseline Ensemble Model

### 42 spectral features
### Recursive Feature Elimination (RFE), GridSearch, Stacked Classifier (SVM, Logistic Regression & Random Forest, with meta-model SVM)

In [ ]:
# Loading data 

directory = '/Flight-Sim-Cognitive-Workload-EEG-Prediction/data'
concatenated_X, concatenated_Y = [], []
for subdir in sorted(os.listdir(directory))[1:]:
    for file in sorted(os.listdir(os.path.join(directory, subdir))):
        # print(directory.split('/')[-1], subdir, file)
        data = np.load(os.path.join(directory, subdir, file))
        X = data['X'][135:] # extract spectral features
        Y = data['Y']
        concatenated_X.append(X)
        concatenated_Y.append(Y)

X = concatenated_X
Y = np.concatenate(concatenated_Y)

In [ ]:
# Recursive Feature Elimination - RFE

num_features = 8
num_folds = 8
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
feature_selection_counts = np.zeros(np.array(X).shape[1])

for k, (train_index, test_index) in enumerate(skf.split(X,Y), start=1):
    X_train, X_test = np.array(X)[train_index], np.array(X)[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    estimator = svm.LinearSVC(max_iter=30000, random_state=42)
    selector = RFE(estimator, n_features_to_select=num_features)
    selector.fit(X_train, Y_train)
    
    feature_selection_counts += selector.support_.astype(int)

feature_selection_frequency = feature_selection_counts / num_folds
best_feature_indices = np.argsort(-feature_selection_frequency)[:num_features]
X_selected = np.array(X)[:, best_feature_indices]

print("Feature selection frequency across folds:", feature_selection_frequency)
print("Best feature indices:", best_feature_indices)

In [ ]:
# hyperparameter tuning - GridSearchCV

scoring = make_scorer(accuracy_score)
num_folds = 8
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

param_grid = {
    # RandomForest parameters
    'rf__n_estimators': [50, 100],
    'rf__max_depth': [None, 10, 20],
    'rf__min_samples_split': [2, 5],
    'rf__min_samples_leaf': [1, 2],
    'rf__bootstrap': [True],
    
    # LogisticRegression parameters
    'lr__C': [0.01, 0.1, 1.0],
    'lr__solver': ['liblinear'],
    
    # SVC (base) parameters
    'svm_base__C': [0.01, 1.0, 10],
    'svm_base__kernel': ['linear'],
    'svm_base__gamma': ['scale', 'auto'],
    
    # Final estimator (meta-model) parameters
    'final_estimator__C': [0.01, 1.0, 10],
    'final_estimator__kernel': ['linear'],
    'final_estimator__gamma': ['scale', 'auto']
}

param_selection_counts = {param: {value: 0 for value in values} for param, values in param_grid.items()}

for k, (train_index, test_index) in enumerate(skf.split(X_selected, Y), start=1):
    X_train, X_test = np.array(X_selected)[train_index], np.array(X_selected)[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    base_classifiers = [
    ('rf', RandomForestClassifier(random_state=42)),
    ('lr', LogisticRegression(max_iter=30000, random_state=42)),
    ('svm_base', svm.SVC(probability=True, random_state=42))
    ]

    meta_model = svm.SVC(random_state=42)

    stacked_classifier = StackingClassifier(
        estimators=base_classifiers,
        final_estimator=meta_model
    )

    stackedmodel_grid_search = GridSearchCV(stacked_classifier, param_grid=param_grid, scoring=scoring, cv=8, verbose=1, n_jobs=-1)
    stackedmodel_grid_search.fit(X_train, Y_train)
    best_stackedmodel_params = stackedmodel_grid_search.best_params_
    
    for param, best_value in best_stackedmodel_params.items():
        param_selection_counts[param][best_value] += 1
    
average_selected_params = {}
for param, value_counts in param_selection_counts.items():
    most_common_value = max(value_counts, key=value_counts.get)
    average_selected_params[param] = most_common_value

print("Most frequently selected hyperparameters across folds:")
for param, value in average_selected_params.items():
    print(f"{param}: {value}")

baseline_model_param = average_selected_params

In [ ]:
# Stacked Classifier

scoring = make_scorer(accuracy_score)
num_folds = 8
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
f1_scores, accuracy_scores, precision_scores, recall_scores, cm_scores = [],[],[],[],[]

base_classifiers = [
    ('rf', RandomForestClassifier(
        n_estimators=average_selected_params['rf__n_estimators'],
        max_depth=average_selected_params['rf__max_depth'],
        min_samples_split=average_selected_params['rf__min_samples_split'],
        min_samples_leaf=average_selected_params['rf__min_samples_leaf'],
        bootstrap=average_selected_params['rf__bootstrap'], 
        random_state=42
    )),
    ('lr', LogisticRegression(
        C=average_selected_params['lr__C'],
        solver=average_selected_params['lr__solver'],
        max_iter=30000, 
        random_state=42
    )),
    ('svm_base', svm.SVC(
        C=average_selected_params['svm_base__C'],
        kernel=average_selected_params['svm_base__kernel'],
        gamma=average_selected_params['svm_base__gamma'],
        probability=True, 
        random_state=42
    ))
    ]

meta_model = svm.SVC(
    C=average_selected_params['final_estimator__C'],
    kernel=average_selected_params['final_estimator__kernel'],
    gamma=average_selected_params['final_estimator__gamma'], 
    random_state=42
)

stacked_classifier = StackingClassifier(
    estimators=base_classifiers,
    final_estimator=meta_model
)
    
for k, (train_index, test_index) in enumerate(skf.split(X_selected, Y), start=1):
    X_train, X_test = np.array(X_selected)[train_index], np.array(X_selected)[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    stacked_classifier.fit(X_train, Y_train)
    Y_pred = stacked_classifier.predict(X_test)
    
    accuracy = accuracy_score(Y_test, Y_pred)
    f1_scores.append(f1_score(Y_test, Y_pred, zero_division=1))
    accuracy_scores.append(accuracy)
    precision_scores.append(precision_score(Y_test, Y_pred, zero_division=1))
    recall_scores.append(recall_score(Y_test, Y_pred, zero_division=1))
    cm_scores.append(confusion_matrix(Y_test, Y_pred))

    print(f'Fold {k} - Accuracy: {round(accuracy, 2)}')

print(' ')
print('Accuracy:', '\n', f'Mean: {round(np.mean(accuracy_scores), 2)}.', f'Std: {round(np.std(accuracy_scores), 2)}.', f'Max: {round(np.max(accuracy_scores), 2)}.', f'Min: {round(np.min(accuracy_scores), 2)}.')
print(' ')
print('F1:', '\n', f'Mean: {round(np.mean(f1_scores), 2)}.', f'Std: {round(np.std(f1_scores), 2)}.', f'Max: {round(np.max(f1_scores), 2)}.', f'Min: {round(np.min(f1_scores), 2)}.')
print(' ')
print('Precision:', '\n', f'Mean: {round(np.mean(precision_scores), 2)}.', f'Std: {round(np.std(precision_scores), 2)}.', f'Max: {round(np.max(precision_scores), 2)}.', f'Min: {round(np.min(precision_scores), 2)}.')
print(' ')
print('Recall:', '\n', f'Mean: {round(np.mean(recall_scores), 2)}.', f'Std: {round(np.std(recall_scores), 2)}.', f'Max: {round(np.max(recall_scores), 2)}.', f'Min: {round(np.min(recall_scores), 2)}.')
print(' ')

combined_cm = np.sum(cm_scores, axis=0)
cm_plot(combined_cm, model='Baseline')

baseline_model_acc = accuracy_scores
baseline_mean = round(np.mean(accuracy_scores), 2)
baseline_std = round(np.std(accuracy_scores), 2)

In [ ]:
# Print features selected by the model

ranked_features = pd.DataFrame(columns=['Frequency_Band', 'Feature_type'])
node_names_orig = ['F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'Cz', 'T8', 'P3', 'Pz', 'P4', 'PO7', 'PO8', 'Oz']
combinations_total = node_names_orig*3
alpha_spec, beta_spec, theta_spec = [], [], []

for i in best_feature_indices:
    new_row = None
    if i < 14:
        alpha_spec.append(combinations_total[i])
        new_row = pd.DataFrame({'Frequency_Band': ['Alpha'], 'Feature_type': [combinations_total[i]]})
    elif i >= 14 and i < 28:
        beta_spec.append(combinations_total[i])
        new_row = pd.DataFrame({'Frequency_Band': ['Beta'], 'Feature_type': [combinations_total[i]]})
    elif i >= 28 and i < 42:
        theta_spec.append(combinations_total[i])
        new_row = pd.DataFrame({'Frequency_Band': ['Theta'], 'Feature_type': [combinations_total[i]]})

    if new_row is not None:
        ranked_features = pd.concat([ranked_features, new_row], ignore_index=True)

print(f'Features ranked order: {ranked_features}')
print(' ')
print(f'Number of chosen spectral features per frequency band: \nTotal: {len(alpha_spec)+len(beta_spec)+len(theta_spec)}')
print(' ')
print(f'Spectral features per frequency band: \nAlpha: {alpha_spec} \nBeta: {beta_spec} \nTheta: {theta_spec}')

BaselineModel_SpectralPower_Chosen = [alpha_spec, beta_spec, theta_spec]
  

# Connectivity Ensemble Model
### 42 spectral features + 30 frontal-parietal PLV (phase-locking value) features
### Recursive Feature Elimination (RFE), GridSearch, Stacked Classifier (SVM, Logistic Regression & Random Forest, with meta-model SVM)

In [ ]:
# Loading data 

directory = '/Flight-Sim-Cognitive-Workload-EEG-Prediction/data'
concatenated_X, concatenated_Y = [], []
for subdir in sorted(os.listdir(directory))[1:]:
    for file in sorted(os.listdir(os.path.join(directory, subdir))):
        # print(directory.split('/')[-1], subdir, file)
        data = np.load(os.path.join(directory, subdir, file))
        X_PLV = fronto_parietal_averagePLV(data['X'][:135]) # Output: 30 bidirectional PLV features (10 Channels x 3 frequency bands)
        X_Freq = data['X'][135:] # spectral power features
        X = np.concatenate((X_PLV, X_Freq))
        Y = data['Y']
        concatenated_X.append(X)
        concatenated_Y.append(Y)

X = concatenated_X
Y = np.concatenate(concatenated_Y)


In [ ]:
# Recursive Feature Elimination - RFE

num_features = 8
num_folds = 8
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
feature_selection_counts = np.zeros(np.array(X).shape[1])

for k, (train_index, test_index) in enumerate(skf.split(X,Y), start=1):
    X_train, X_test = np.array(X)[train_index], np.array(X)[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    estimator = svm.LinearSVC(max_iter=30000, random_state=42)
    selector = RFE(estimator, n_features_to_select=num_features)
    selector.fit(X_train, Y_train)
    
    feature_selection_counts += selector.support_.astype(int)

feature_selection_frequency = feature_selection_counts / num_folds
best_feature_indices = np.argsort(-feature_selection_frequency)[:num_features]
X_selected = np.array(X)[:, best_feature_indices]

print("Feature selection frequency across folds:", feature_selection_frequency)
print("Best feature indices:", best_feature_indices)

In [ ]:
# hyperparameter tuning - GridSearchCV

scoring = make_scorer(accuracy_score)
num_folds = 8
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

param_grid = {
    # RandomForest parameters
    'rf__n_estimators': [50, 100],
    'rf__max_depth': [None, 10, 20],
    'rf__min_samples_split': [2, 5],
    'rf__min_samples_leaf': [1, 2],
    'rf__bootstrap': [True],
    
    # LogisticRegression parameters
    'lr__C': [0.01, 0.1, 1.0],
    'lr__solver': ['liblinear'],
    
    # SVC (base) parameters
    'svm_base__C': [0.01, 1.0, 10],
    'svm_base__kernel': ['linear'],
    'svm_base__gamma': ['scale', 'auto'],
    
    # Final estimator (meta-model) parameters
    'final_estimator__C': [0.01, 1.0, 10],
    'final_estimator__kernel': ['linear'],
    'final_estimator__gamma': ['scale', 'auto']
}

param_selection_counts = {param: {value: 0 for value in values} for param, values in param_grid.items()}

for k, (train_index, test_index) in enumerate(skf.split(X_selected, Y), start=1):
    X_train, X_test = np.array(X_selected)[train_index], np.array(X_selected)[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    base_classifiers = [
    ('rf', RandomForestClassifier(random_state=42)),
    ('lr', LogisticRegression(max_iter=30000, random_state=42)),
    ('svm_base', svm.SVC(probability=True, random_state=42))
    ]

    meta_model = svm.SVC(random_state=42)

    stacked_classifier = StackingClassifier(
        estimators=base_classifiers,
        final_estimator=meta_model
    )

    stackedmodel_grid_search = GridSearchCV(stacked_classifier, param_grid=param_grid, scoring=scoring, cv=8, verbose=1, n_jobs=-1)
    stackedmodel_grid_search.fit(X_train, Y_train)
    best_stackedmodel_params = stackedmodel_grid_search.best_params_
    
    for param, best_value in best_stackedmodel_params.items():
        param_selection_counts[param][best_value] += 1
    
average_selected_params = {}
for param, value_counts in param_selection_counts.items():
    most_common_value = max(value_counts, key=value_counts.get)
    average_selected_params[param] = most_common_value

print("Most frequently selected hyperparameters across folds:")
for param, value in average_selected_params.items():
    print(f"{param}: {value}")

connectivity_model_param = average_selected_params

In [ ]:
# Stacked Classifier

scoring = make_scorer(accuracy_score)
num_folds = 8
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
f1_scores, accuracy_scores, precision_scores, recall_scores, cm_scores = [],[],[],[],[]

base_classifiers = [
    ('rf', RandomForestClassifier(
        n_estimators=average_selected_params['rf__n_estimators'],
        max_depth=average_selected_params['rf__max_depth'],
        min_samples_split=average_selected_params['rf__min_samples_split'],
        min_samples_leaf=average_selected_params['rf__min_samples_leaf'],
        bootstrap=average_selected_params['rf__bootstrap'],
        random_state=42
    )),
    ('lr', LogisticRegression(
        C=average_selected_params['lr__C'],
        solver=average_selected_params['lr__solver'],
        max_iter=30000,
        random_state=42
    )),
    ('svm_base', svm.SVC(
        C=average_selected_params['svm_base__C'],
        kernel=average_selected_params['svm_base__kernel'],
        gamma=average_selected_params['svm_base__gamma'],
        probability=True,
        random_state=42
    ))
    ]

meta_model = svm.SVC(
    C=average_selected_params['final_estimator__C'],
    kernel=average_selected_params['final_estimator__kernel'],
    gamma=average_selected_params['final_estimator__gamma'],
    random_state=42
)

stacked_classifier = StackingClassifier(
    estimators=base_classifiers,
    final_estimator=meta_model
)
    
for k, (train_index, test_index) in enumerate(skf.split(X_selected, Y), start=1):
    X_train, X_test = np.array(X_selected)[train_index], np.array(X_selected)[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    stacked_classifier.fit(X_train, Y_train)
    Y_pred = stacked_classifier.predict(X_test)
    
    accuracy = accuracy_score(Y_test, Y_pred)
    f1_scores.append(f1_score(Y_test, Y_pred, zero_division=1))
    accuracy_scores.append(accuracy)
    precision_scores.append(precision_score(Y_test, Y_pred, zero_division=1))
    recall_scores.append(recall_score(Y_test, Y_pred, zero_division=1))
    cm_scores.append(confusion_matrix(Y_test, Y_pred))

    print(f'Fold {k} - Accuracy: {round(accuracy, 2)}')

print(' ')
print('Accuracy:', '\n', f'Mean: {round(np.mean(accuracy_scores), 2)}.', f'Std: {round(np.std(accuracy_scores), 2)}.', f'Max: {round(np.max(accuracy_scores), 2)}.', f'Min: {round(np.min(accuracy_scores), 2)}.')
print(' ')
print('F1:', '\n', f'Mean: {round(np.mean(f1_scores), 2)}.', f'Std: {round(np.std(f1_scores), 2)}.', f'Max: {round(np.max(f1_scores), 2)}.', f'Min: {round(np.min(f1_scores), 2)}.')
print(' ')
print('Precision:', '\n', f'Mean: {round(np.mean(precision_scores), 2)}.', f'Std: {round(np.std(precision_scores), 2)}.', f'Max: {round(np.max(precision_scores), 2)}.', f'Min: {round(np.min(precision_scores), 2)}.')
print(' ')
print('Recall:', '\n', f'Mean: {round(np.mean(recall_scores), 2)}.', f'Std: {round(np.std(recall_scores), 2)}.', f'Max: {round(np.max(recall_scores), 2)}.', f'Min: {round(np.min(recall_scores), 2)}.')
print(' ')

combined_cm = np.sum(cm_scores, axis=0)
cm_plot(combined_cm, model='Connectivity_Model')

connectivity_model_acc = accuracy_scores
connectivity_mean = round(np.mean(accuracy_scores), 2)
connectivity_std = round(np.std(accuracy_scores), 2)

In [ ]:
# Print features selected by the model

ranked_features = pd.DataFrame(columns=['Frequency_Band_&_Feature_set', 'Feature_type'])
plv_names = ['F7', 'F3', 'Fz', 'F4', 'F8', 'P3', 'Pz', 'P4', 'PO7', 'PO8']
node_names_orig = ['F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'Cz', 'T8', 'P3', 'Pz', 'P4', 'PO7', 'PO8', 'Oz']
combinations_total = plv_names*3 + node_names_orig*3

alpha_plv, beta_plv, theta_plv = [], [], []
alpha_spec, beta_spec, theta_spec = [], [], []

for i in best_feature_indices:
    new_row = None
    if i < 10:
        alpha_plv.append(combinations_total[i])
        new_row = pd.DataFrame({'Frequency_Band_&_Feature_set': ['Alpha-PLV'], 'Feature_type': [combinations_total[i]]})
    elif i >= 10 and i < 20:
        beta_plv.append(combinations_total[i])
        new_row = pd.DataFrame({'Frequency_Band_&_Feature_set': ['Beta-PLV'], 'Feature_type': [combinations_total[i]]})
    elif i >= 20 and i < 30:
        theta_plv.append(combinations_total[i])
        new_row = pd.DataFrame({'Frequency_Band_&_Feature_set': ['Theta-PLV'], 'Feature_type': [combinations_total[i]]})
    elif i >= 30 and i < 44:
        alpha_spec.append(combinations_total[i])
        new_row = pd.DataFrame({'Frequency_Band_&_Feature_set': ['Alpha-Spectral'], 'Feature_type': [combinations_total[i]]})
    elif i >= 44 and i < 58:
        beta_spec.append(combinations_total[i])
        new_row = pd.DataFrame({'Frequency_Band_&_Feature_set': ['Beta-Spectral'], 'Feature_type': [combinations_total[i]]})
    elif i >= 58 and i < 72:
        theta_spec.append(combinations_total[i])
        new_row = pd.DataFrame({'Frequency_Band_&_Feature_set': ['Theta-Spectral'], 'Feature_type': [combinations_total[i]]})

    if new_row is not None:
        ranked_features = pd.concat([ranked_features, new_row], ignore_index=True)

print(f'Ranked order: \n {ranked_features}')        
print(' ')
print('Number of Connectivity features chosen: ', len(alpha_plv) + len(beta_plv) + len(theta_plv))
print(f'PLV features per frequency band: \nAlpha: {alpha_plv} \nBeta: {beta_plv} \nTheta: {theta_plv}')
print(' ')
print('Number of Spectral features chosen: ', len(alpha_spec) + len(beta_spec) + len(theta_spec))
print(f'Spectral features per frequency band: \nAlpha: {alpha_spec} \nBeta: {beta_spec} \nTheta: {theta_spec}')

Connectivity_Chosen = [alpha_plv, beta_plv, theta_plv]
ConnectivityModel_SpectralPower_Chosen = [alpha_spec, beta_spec, theta_spec]

# Box plot and Results summary

In [ ]:
# Boxplot accuracy-scores per model.

data = [baseline_model_acc, connectivity_model_acc]
sns.set(style="white")
palette = ["lightblue"]

plt.figure(figsize=(8, 6))
sns.boxplot(data=data, palette=palette, linewidth=2, boxprops={"edgecolor": "darkblue"}, medianprops={"color": "darkblue"}, whiskerprops={"color": "darkblue"}, capprops={"color": "darkblue"})
plt.xticks(ticks=range(len(['Baseline Model', 'Connectivity Model'])),labels=['Baseline Model', 'Connectivity Model'])
plt.ylabel("Accuracy")
plt.ylim([0,1])

# Add median scores on the y-axis
medians = [np.median(baseline_model_acc), np.median(connectivity_model_acc)]
for i, median in enumerate(medians):
    plt.text(i, median, f'{median:.2f}', horizontalalignment='center', verticalalignment='bottom', fontdict={'weight': 'bold', 'size': 12}, color='darkblue')

plt.title(f'Mean&Std Baseline {baseline_mean, baseline_std}, Mean&Std Connectivity-model {connectivity_mean, connectivity_std}')
sns.despine()
plt.show()

output_file = 'Flight-Sim-Cognitive-Workload-EEG-Prediction/results/boxplot_accuracy.png'
plt.savefig(output_file, bbox_inches='tight')
plt.clf()


In [ ]:
# Selected spectral features and PLV features per model.

SpectralPower_Chosen = BaselineModel_SpectralPower_Chosen, ConnectivityModel_SpectralPower_Chosen
models = ['Baseline Model', 'Connectivity Model']
for name, selected in zip(models, SpectralPower_Chosen):
    print(f'Selected spectral features of {name}: {selected}')

name = 'Connectivity Model'
print(f'Selected Connectivity (PLV) features of {name}: {Connectivity_Chosen}') 

In [ ]:
# Print model with highest accuracy among folds

acc_scores = baseline_model_acc, connectivity_model_acc
chosen_param = baseline_model_param, connectivity_model_param

new_list = []
for i in acc_scores:
    idx = np.argmax(i)
    new_list.append(i[idx])
idx1 = np.argmax(new_list)

if idx1 == 0:
    model = 'Baseline Model' 
elif idx1 == 1:
    model = 'Connectivity Model'
print(f'Best performing model based on highest accuracy among folds: {model} \n max accuracy score: {new_list[idx1]}. \n mean accuracy score: {np.mean(acc_scores[idx1])}. \n best performing set of hyperparameters: {chosen_param[idx1]}.')

In [ ]:
# Print best performing model based on highest mean accuracy over folds.

mean_acc_scores = np.mean(acc_scores, axis=1)
idx = np.argmax(mean_acc_scores)
idx1 = np.argmax(acc_scores[idx])
if idx == 0:
    model = 'Baseline Model'
elif idx == 1:
    model = 'Connectivity Model'
print(f'Best performing model based on highest mean accuracy: {model} \n max accuracy score: {acc_scores[idx][idx1]}. \n mean accuracy score: {np.mean(mean_acc_scores[idx])}. \n best performing set of hyperparameters: {chosen_param[idx]}.')